In [1]:
#general & system\n",
import os

#data processing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = 10, 10  #default setting
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

#data augmentation
from PIL import Image
from random import choice
import cv2
import numpy as np
#import keras.preprocessing.image as prep

#ML part
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing import image as kimage
from keras.models import Sequential
from keras.layers import GlobalMaxPooling2D, Dense, BatchNormalization, GlobalAveragePooling2D,Dropout,Activation,Flatten
from keras.models import Model
from keras.optimizers import SGD
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator

#validation
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve, auc,classification_report,roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve, auc,classification_report,roc_auc_score


from sklearn.metrics import precision_recall_curve
from imblearn.over_sampling import SMOTE
from sklearn.utils import resample

%matplotlib inline

Using TensorFlow backend.


In [2]:
%%time
meta = pd.read_csv("../data/label_learn.csv", sep = ";")

CPU times: user 4 ms, sys: 4 ms, total: 8 ms
Wall time: 5.99 ms


In [3]:
meta.head()

,name,label
0,0000000.jpg,benign
1,0000002.jpg,malignant
2,0000006.jpg,benign
3,0000008.jpg,benign
4,0000009.jpg,benign


In [4]:
%%time
train_list = os.listdir("../data/resized_train//")
len(train_list) == len(meta.name)

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 3.86 ms


In [5]:
%%time
X_train = np.array([np.array(Image.open("../data/resized_train/"+fname)) for fname in meta.name])
Y_train = [1 if x=="malignant" else 0 for x in meta.label]

CPU times: user 22 s, sys: 1.69 s, total: 23.7 s
Wall time: 23.7 s


# Data Preparation 

## Hair removal : 


We proceed following the steps :

- Data acquisition : Select a subset of data with Hair
- RGB to Grayscale conversion – Contrast Enhancement: Convert to Gray scale
- Binarization : Binarize the Image : Convert the image to binary using an adaptive threshold
- Edge Detection : Using Canny edge detector algorithm
- Removal non-hair edges : 
- Hair mask creation : 



## Data split for validation 

In [6]:
X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [7]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=2, random_state=None, shuffle=False)
Data_sets = []
for train_index, test_index in skf.split(X_train, Y_train):
    x_train, x_valid = X_train[train_index], X_train[test_index]
    y_train, y_valid = Y_train[train_index], Y_train[test_index]
    Data_sets.append([x_train, x_valid, y_train, y_valid])

## Resampling

In [8]:
X_train = np.array(Data_sets[0][0])
Y_train = np.array(Data_sets[0][2])

In [9]:
temp = sum(Y_train)
sample_size = temp*2700/(374+254)  #1372
sample_size
def random_undersampling(X, Y,sample_size = sample_size, n_sample=1):
    """
    X,y : numpy arrays
    return :
    5 random ensemble of indices general_balenced_set:
        general_balenced_set[0] = the shuffeled indices that inssure the class balance
    """
    indices = np.array(range(len(Y)))
    positive_samples = indices[Y==1]
    #print(type(positive_samples))
    negative_samples = indices[Y==0]
    #print(type(negative_samples))
    general_balenced_set = []
    for k in range(n_sample):
        indices_ = np.random.choice(negative_samples, sample_size, replace=False)
        #print(len(indices_))
        # append positive and negative
        balenced_set = np.append(indices_, positive_samples)
        #print(len(balenced_set))
        # shuffle indices
        np.random.shuffle(balenced_set)
        #print(len(balenced_set))
        general_balenced_set.append(balenced_set)
    return general_balenced_set

In [10]:
general_balenced_set = random_undersampling(X_train,Y_train)
X_train = X_train[general_balenced_set]
Y_train = Y_train[general_balenced_set]

# Loss Function 

In [11]:
from keras import backend as K

def weighted_categorical_crossentropy(weights):
    """
    [w_0, w_1] : weight of 0 class and weight of 1 class
    """
    weights = K.variable(weights)

    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        #y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights[1] + (1 - y_true) * K.log(1 - y_pred) * weights[0]
        loss = -K.sum(loss, -1)
        return loss

    return loss

In [12]:
class_weight_or = np.array([1.8,4])

In [22]:
loss1 = weighted_categorical_crossentropy(np.array([1.8,6]))

# Metrics to print 

In [14]:
def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())



# Define  Deep Learing Model

In [15]:
weights_path = '../model/checkpoints/'
    
if not os.path.exists(weights_path):
    os.makedirs(weights_path)
    
epochs = 500
batch_size = 50

#load base model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(299,299,3))
#adding top layers
    #sequentialy
add_model = Sequential()
add_model.add(GlobalAveragePooling2D(input_shape=base_model.output_shape[1:])) #Flatten/GlobalAveragePooling2D
add_model.add(Dense(1024, activation='relu'))
add_model.add(Dropout(0.25))
add_model.add(Dense(512, activation='relu'))
add_model.add(Dropout(0.25))
add_model.add(Dense(1, activation='sigmoid'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))


#freeze lower layers of the model
#for layer in model.layers[:]:
    #layer.trainable = False
for layer in model.layers[0:]:
    layer.trainable = True

# Compile the model

In [16]:
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss=loss1, #binary_crossentropy
              metrics=[sensitivity, specificity,"accuracy"])

# Cross-validation of the model 

In [18]:
%%time
#Lets define the image transormations that we wan

gen = ImageDataGenerator(horizontal_flip=True,
                         vertical_flip=True,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         zoom_range=0.2,
                         rotation_range=40)

val_datagen = ImageDataGenerator()
# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_one_input(X1, y):
    genX1 = gen.flow(X1, y, batch_size=batch_size, seed=42)
    while True:
        X1i = genX1.next()
        yield X1i[0], X1i[1]

def val_datagen_(X1, y):
    genX1 = val_datagen.flow(X1, y, batch_size=batch_size, seed=42)
    while True:
        X1i = genX1.next()
        yield X1i[0], X1i[1]
#Finally create out generator
gen_flow_train = gen_flow_for_one_input(X_train, Y_train)
#gen_flow_val = gen_flow_for_one_input(X_valid, Y_valid)
gen_flow_val = val_datagen_(Data_sets[0][1], Data_sets[0][3])


CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 72 µs


In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
sensitivity = []
specificty = []
accuracy = []

for [X_train, X_valid, y_train, y_valid] in Data_sets: 
    gen_flow_train = gen_flow_for_one_input(X_train, y_train)
    epochs_to_wait_for_improve = 5
    early_stopping_callback = EarlyStopping(monitor='val_loss', patience=epochs_to_wait_for_improve)
    checkpoint_callback = ModelCheckpoint(weights_path + 'BestKerasResnet50_flat_3_loss.h5', monitor='val_loss',
                                          verbose=1, save_best_only=True, mode='min')
    #fit the model
    model.fit_generator(gen_flow_train, validation_data=(X_valid, y_valid),
                        steps_per_epoch=int(np.ceil(len(X_train)/batch_size)),
                        epochs=500, verbose=1, callbacks=[early_stopping_callback, checkpoint_callback],
                        class_weight = class_weight_or)

    #evaluate the model
    scores = model.evaluate(X_valid, y_valid, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    print("%s: %.2f%%" % (model.metrics_names[2], scores[2]*100))
    sensitivity.append(scores[0] * 100)
    specificity.append(scores[1] * 100)
    accuracy.append(scores[2] * 100)

Epoch 1/500
85/85 [==============================] - 87s 1s/step - loss: 0.3779 - sensitivity: 0.6545 - specificity: 0.9668 - acc: 0.9409 - val_loss: 0.6205 - val_sensitivity: 0.0341 - val_specificity: 0.9519 - val_acc: 0.9333
Epoch 2/500
43/85 [==============>...............] - ETA: 29s - loss: 0.3455 - sensitivity: 0.7532 - specificity: 0.9558 - acc: 0.9377